In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [6]:
# train 데이터 불러오기
train = pd.read_csv('/Users/jian/AIVLE/AI_SPARK/dataset/train_data.csv')

# test 데이터 불러오기
test = pd.read_csv('/Users/jian/AIVLE/AI_SPARK/dataset/test_data.csv')

result = pd.read_csv('/Users/jian/AIVLE/AI_SPARK/dataset/answer_sample.csv')

In [11]:
from keras.layers.rnn import TimeDistributed
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.layers import  Input, Dense, Reshape, Layer, Lambda, RepeatVector, Concatenate, Dot, Activation,Flatten
from keras.layers import Multiply, LSTM, Permute, Conv1D
from keras.models import Model
from keras import regularizers
from keras.backend import clear_session
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

In [12]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   verbose=1,
                   patience=10,
                   restore_best_weights=True
                   )
mc = ModelCheckpoint('./model.h5',
                     verbose=0,
                     monitor='val_loss',
                     save_best_only=True,
                     save_best_weights=True)


In [13]:
def ae_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L = Conv1D(filters=512, 
               kernel_size=64, 
               padding='same', 
               data_format='channels_last',
               dilation_rate=1, 
               activation="linear")(inputs)
    L0 = Dense(128)(L)
    
    # Add attention mechanism
    e = Dense(X.shape[1], activation='tanh')(L0)
    e = Dense(1, activation='sigmoid')(e)
    att_rep = Permute([2, 1])(e)
    att_l = Multiply()([L0, att_rep])

    L1 = LSTM(64, activation='relu', return_sequences=False)(att_l)
    L2 = Dense(64)(L1)
    encoded = RepeatVector(X.shape[1])(L2)
    L3 = Dense(64)(encoded)


    # Add attention mechanism
    e = Dense(X.shape[1], activation='tanh')(L3)
    e = Dense(1, activation='sigmoid')(e)
    att_rep = Permute([2, 1])(e)
    att_l = Multiply()([L3, att_rep])

    L4 = LSTM(64, activation='relu', return_sequences=True)(att_l)
    L5 = Dense(128)(L4)
    L6 = Conv1D(filters=512, 
               kernel_size=64, 
               padding='same', 
               data_format='channels_last',
               dilation_rate=1, 
               activation="linear")(L5)
    decoded = Dense(X.shape[2], activation=None)(L6)

    model = Model(inputs, decoded)

    return model

In [15]:
def modeling(type_num):
    clear_session()
    
    if type_num == -1:
        train_ = train[train['type'].isin([0, 4, 5, 6, 7])].iloc[:, :-1]
        test_ = test[test['type'].isin([0, 4, 5, 6, 7])].iloc[:, :-1]
    else:
        train_ = train[train['type'] == type_num].iloc[:, :-1]
        test_ = test[test['type'] == type_num].iloc[:, :-1]

    # 정규화
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_)
    test_scaled = scaler.transform(test_)

    # reshape
    train_scaled = train_scaled.reshape(train_scaled.shape[0], 1, train_scaled.shape[1])
    test_scaled = test_scaled.reshape(test_scaled.shape[0], 1, test_scaled.shape[1])

    # 모델 컴파일
    model = ae_model(train_scaled)
    model.compile(optimizer=Adam(lr=0.001), loss='mse')

    # 모델 학습
    history = model.fit(train_scaled, train_scaled, epochs=100, batch_size=32, validation_split=0.1, callbacks=[es,mc]).history

    # train 재구성 오차를 계산합니다.
    train_pred = model.predict(train_scaled)
    train_pred = train_pred.reshape(train_pred.shape[0], train_pred.shape[2])
    train_pred = pd.DataFrame(train_pred, columns=train_.columns)
    train_pred.index = train_.index

    scored = pd.DataFrame(index=train_.index)
    Xtrain = train_scaled.reshape(train_scaled.shape[0], train_scaled.shape[2])
    scored['Loss_mse'] = np.mean(np.abs(train_pred - Xtrain), axis=1)


    # test 재구성 오차
    threshold = np.percentile(scored, 100)     

    test_pred = model.predict(test_scaled)
    test_pred = test_pred.reshape(test_pred.shape[0], test_pred.shape[2])
    test_pred = pd.DataFrame(test_pred, columns=test_.columns)
    test_pred.index = test_.index

    test_scored = pd.DataFrame(index=test_.index)
    Xtest = test_scaled.reshape(test_scaled.shape[0], test_scaled.shape[2])
    test_scored['Loss_mse'] = np.mean(np.abs(test_pred - Xtest), axis=1)
    test_scored['Threshold'] = threshold
    test_scored['Anomaly'] = test_scored['Loss_mse'] > test_scored['Threshold']

    test_scored['Anomaly'] = test_scored['Anomaly'].replace(False, 0)
    test_scored['Anomaly'] = test_scored['Anomaly'].replace(True, 1)
    test_scored.head()

    print(test_scored['Anomaly'].value_counts())
    
    if type_num == -1:
        result_ = result[result['type'].isin([0, 4, 5, 6, 7])]
        result_['label'] = test_scored['Anomaly'].values
    else:
        result_ = result[result['type'] == type_num]
        result_['label'] = test_scored['Anomaly'].values
        
    
    return result_

## 결과 저장

In [16]:
result_1 = modeling(1)
result_2 = modeling(2)
result_3 = modeling(3)
result_0 = modeling(-1)

Epoch 1/100
11/11 [==============================] - 11s 515ms/step - loss: 0.1563 - val_loss: 0.0533
Epoch 2/100
11/11 [==============================] - 5s 441ms/step - loss: 0.0625 - val_loss: 0.0507
Epoch 3/100
11/11 [==============================] - 5s 469ms/step - loss: 0.0444 - val_loss: 0.0237
Epoch 4/100
11/11 [==============================] - 5s 470ms/step - loss: 0.0163 - val_loss: 0.0075
Epoch 5/100
11/11 [==============================] - 6s 588ms/step - loss: 0.0098 - val_loss: 0.0031
Epoch 6/100
11/11 [==============================] - 5s 435ms/step - loss: 0.0074 - val_loss: 0.0015
Epoch 7/100
11/11 [==============================] - 5s 436ms/step - loss: 0.0062 - val_loss: 3.5639e-04
Epoch 8/100
11/11 [==============================] - 6s 523ms/step - loss: 0.0059 - val_loss: 2.8166e-04
Epoch 9/100
11/11 [==============================] - 5s 414ms/step - loss: 0.0059 - val_loss: 4.2807e-04
Epoch 10/100
11/11 [==============================] - 5s 424ms/step - loss: 0.

In [17]:
final = pd.concat([result_0, result_1], axis=0).sort_index()
final = pd.concat([final, result_2], axis=0).sort_index()
final = pd.concat([final, result_3], axis=0).sort_index()

In [18]:
final['label'].value_counts()

0    7042
1     347
Name: label, dtype: int64

In [ ]:
final.to_csv('./autoencoder_attention_v3.csv', index=False)